In [5]:
!pip install kaggle

In [6]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer # mapping the words to certain integers
from tensorflow.keras.preprocessing.sequence import pad_sequences # to insure all the input data are of the same shape

In [7]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [8]:
import os
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [9]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 19% 5.00M/25.7M [00:00<00:00, 47.8MB/s]
100% 25.7M/25.7M [00:00<00:00, 136MB/s] 


In [10]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
    zip_ref.extractall()


In [11]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [12]:
data = pd.read_csv("IMDB Dataset.csv")

In [13]:
data.shape

(50000, 2)

In [14]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [15]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [16]:
# converting +ve to 1 and -ve to 0

data.replace({"sentiment": {"positive":1, "negative":0}}, inplace=True)

<ipython-input-16-aeaab87f5c8f>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive":1, "negative":0}}, inplace=True)


In [17]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [18]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [19]:
# preprocessing, includes tokenizing the data, feature vector conversion
# tokenizer step:
# tokenizing means to map every unique words to a number, like "Bad" to 3 etc

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"]) # first we are mapping all the words to some unique integer
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200) # now we are making an vector of sequence of the original sentence.
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200) # along with the vector we are adding padding to the data


In [20]:
X_train[0]

array([1935,    1, 1200,    4, 2946, 3749, 1828,    2,  147,  144,    3,
        228,    4,    3,  207,  323,    2,  144, 1083,   16,   88,    4,
        132, 2871,   18,   10,  153,   99,    4,    1, 4020,  302,   11,
         17, 1001,   35,    1,  496,  492, 2619,  249,   71,   77,  107,
        107,  698,   60,   86, 1047, 1363,    5,  229,  132,   23, 4360,
         31,  138,  209, 1154,   14, 4501,   31,    3, 2386,    2,    8,
         11,    6,    3,  445,   14,  624,    4,    1,  718, 2959,    1,
       1278,    2,   71, 3616,    1,  166, 1507,    1, 1245,    5, 1629,
          1,  879, 1268,    5,    1,  310,  140, 2894,    2,  410,  633,
          7,    7,    1,  269,    6, 3553, 1000,    5,   26,   39,   14,
       1381,  217,   65,    2,   46,    6,   30,  219,   27,  193, 1484,
          8, 1101,   18,   10, 4905,   84,    1,  226,   66,  356,   68,
         54,   27,    5, 3600,   15,   44,   21,  192,    5,    3,  889,
       3511, 1758,   22,   25,    5,  158,  196,  1

In [21]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [27]:
# building the LSTM model, this is a type of RNN
# useful for datasets like timeseries where the current datapoint is linked to previous datapoint
# for movie reviews, it shoudn't look at all words separately but rather as a sequence

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128))
# this 5000 is the same as the tokenizer num_words
# output is ..
# input_length as 200 is the same as our pad_sequence, this is similar to CNN where we are saying the input_dim of the image such as 28x28
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) # u can think of this 128 as the neorons.
model.add(Dense(1, activation="sigmoid"))

In [28]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [31]:
model.fit(X_train, Y_train, batch_size=64, validation_split=0.2, epochs=10)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 207s 399ms/step - accuracy: 0.7342 - loss: 0.5226 - val_accuracy: 0.8489 - val_loss: 0.3599
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 265s 414ms/step - accuracy: 0.8613 - loss: 0.3453 - val_accuracy: 0.8381 - val_loss: 0.3642
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 258s 406ms/step - accuracy: 0.8808 - loss: 0.3004 - val_accuracy: 0.7959 - val_loss: 0.4204
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 257s 397ms/step - accuracy: 0.8760 - loss: 0.3049 - val_accuracy: 0.8579 - val_loss: 0.3859
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 196s 385ms/step - accuracy: 0.9014 - loss: 0.2493 - val_accuracy: 0.8664 - val_loss: 0.3272
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 383ms/step - accuracy: 0.9079 - loss: 0.2301 - val_accuracy: 0.8763 - val_loss: 0.3205
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 384ms/step - accuracy: 0.9331 - loss: 0.1822 - val_accuracy: 0.8572 - val_loss: 0.3502
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 196s 373ms/step - accuracy: 0.8966 -

In [32]:
loss, accuracy = model.evaluate(X_test, Y_test)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 109ms/step - accuracy: 0.8700 - loss: 0.3712
Loss: 0.36593881249427795
Accuracy: 0.8736000061035156


In [37]:
# single prediction system

def predict(review):
  sequence = tokenizer.texts_to_sequences([review]) # this tokenizer is taking an array of texts.
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  return "positive" if prediction > 0.5 else "negative"

In [50]:
example_review = "The movie had alot of twists and turns. worse"

print(f"The sentiment of the review is: {predict(example_review)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
The sentiment of the review is: negative
